In [17]:
from math import sqrt
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pydataset import data
import statistics
import acquire
import prepare

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import explore
import warnings
warnings.filterwarnings("ignore")

import graphviz
from graphviz import Graph

### Using the titanic data, in your classification-exercises repository, create a notebook, model.ipynb where you will do the following:

In [7]:
train, validate, test = prepare.prep_titanic_data(acquire.get_titanic_data())

In [9]:
train.head(3)

,passenger_id,survived,pclass,sex,sibsp,parch,fare,embark_town,alone,sex_female,sex_male,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
455,455,1,3,male,0,0,7.8958,Cherbourg,1,0,1,1,0,0
380,380,1,1,female,0,0,227.5250,Cherbourg,1,1,0,1,0,0
492,492,0,1,male,0,0,30.5000,Southampton,1,0,1,0,0,1


In [10]:
train.drop(columns=['sex', 'embark_town'], inplace=True)
validate.drop(columns=['sex', 'embark_town'], inplace=True)
test.drop(columns=['sex', 'embark_town'], inplace=True)

### What is your baseline prediction?

In [11]:
# create X & y version of train, where y is a series with just the target variable and X are all the features. 

X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived

In [12]:
clf = DecisionTreeClassifier(max_depth=3, random_state=123)

In [13]:
clf = clf.fit(X_train, y_train)

In [14]:
y_pred = clf.predict(X_train)
y_pred[0:5]

array([0, 1, 0, 0, 0])

In [15]:
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba[0:5]

array([[0.90322581, 0.09677419],
       [0.        , 1.        ],
       [0.61458333, 0.38541667],
       [0.86666667, 0.13333333],
       [0.90322581, 0.09677419]])

In [16]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.81


In [19]:
dot_data = export_graphviz(clf, feature_names= X_train.columns,class_names=['died','lived'], rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data) 

graph.render('titanic_decision_tree', view=True)

'titanic_decision_tree.pdf'

### What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode).

### When you make those predictions, what is your accuracy? This is your baseline accuracy.